In [17]:
import os, re, glob
import pandas as pd
import numpy as np

pd.options.display.max_columns = 140
print("pandas", pd.__version__)

pandas 2.3.2


## Locate input files & infer (year, group)

In [25]:
# point to your folder with the .sav files
FOLDER = r"D:\MontanaBackup\WolfData\WolfDatasets"

file_names = [
    "Final Respondents Deer Elk License Holders Wolf Survey 2016.csv",
    "Final Respondents Private Landowner Wolf Survey 2016.csv",
    "Final Respondents Wolf License Holders Wolf Survey 2016.csv",
    "Full Household Results 2017.csv",
    "Households 2012 Wolf Hunt Survey Respondents.csv",
    "FINAL Quantitative Respondent Data for Wolf Deer Elk Hunter Survey 2023.csv",
    "FINAL Quantitative Respondent Data for Wolf Hunter Wolf Survey 2023.csv",
    "FINAL Quantitative Respondent Data for Wolf Private Landowner Survey 2023.csv",
    "FINAL Quantitative Respondent Data for the Wolf Household Survey 2023.csv",
    "Resident Deer Elk License Holders 2012 Wolf Hunt Survey Results.csv",
    "Resident Private Landowners 2012 Wolf Hunt Survey Results.csv",
    "Resident Wolf License Holders 2012 Wolf Hunt Survey Results.csv",
]


## Define canonical schemas (per cohort)

In [31]:
# ==== Canonical House23 column order (target layout) ====
HOUSE23 = ['IDNO','Q1a','Q1b','Q1c','Q1d','Q1e','Q1f',
           'Q2a','Q2b','Q2c','Q2d','Q2e',
           'Q3','Q4','Q5','Q6','Q7a','Q7b',
           'Q8a','Q8b','Q9','Q10','Q11a','Q11b',
           'Q12','Q13','Q14','Q15a','Q15b','Q15c',
           'Q15d','Q15e','Q16a','Q16b','Q16c','Q16d',
           'Q17','Q17a1','Q17a2','Q17a3','Q17a4','Q17a5',
           'Q17a6','Q17a7','Q17a8','Q17b','Q18','Q19',
           'Q20','Q21','Q22a','Q22b','Q22c','Q22d','Q22e',
           'Q22f','Q23a','Q23b','Q23c','Q24','Q25',
           'Q26a','Q26b','Q27a','Q27b','Q27c','Q27d',
           'Q27e','Q27f','Q27g','Q27h','Q27i','Q27j',
           'Q27k','Q27l','Q27m','Q28','Q29','Q30','Q31',
           'Q32','Q33','Q34','Q35a','Q35b','Q36','Q37','wt']

# ==== PASTE YOUR SCHEMAS BELOW ====

HOUSE23_SCHEMA = ['IDNO','Q1a', 'Q1b', 'Q1c', 'Q1d', 'Q1e', 'Q1f',
              'Q2a', 'Q2b', 'Q2c', 'Q2d', 'Q2e',
              'Q3', 'Q4', 'Q5', 'Q6', 'Q7a', 'Q7b',
              'Q8a', 'Q8b', 'Q9', 'Q10', 'Q11a', 'Q11b',
              'Q12', 'Q13', 'Q14', 'Q15a', 'Q15b', 'Q15c',
              'Q15d', 'Q15e', 'Q16a', 'Q16b', 'Q16c', 'Q16d',
              'Q17', 'Q17a1', 'Q17a2', 'Q17a3', 'Q17a4', 'Q17a5',
              'Q17a6', 'Q17a7', 'Q17a8', 'Q17b', 'Q18', 'Q19',
              'Q20', 'Q21', 'Q22a', 'Q22b', 'Q22c', 'Q22d', 'Q22e',
              'Q22f', 'Q23a', 'Q23b', 'Q23c', 'Q24', 'Q25',
              'Q26a', 'Q26b', 'Q27a', 'Q27b', 'Q27c', 'Q27d',
              'Q27e', 'Q27f', 'Q27g', 'Q27h', 'Q27i', 'Q27j',
              'Q27k', 'Q27l', 'Q27m', 'Q28', 'Q29', 'Q30', 'Q31',
              'Q32', 'Q33', 'Q34', 'Q35a', 'Q35b', 'Q36', 'Q37','wt']

DEER23_SCHEMA = ['IDNO','Q1a', 'Q1b', 'Q1c', 'Q1d', 'Q1e', 'Q1f',
             'Q2a', 'Q2b', 'Q2c', 'Q2d', 'Q2e',
             'Q3', 'Q4', 'Q5', 'Q6', 'Q7a', 'Q7b',
             'Q8a', 'Q8b', 'Q9', 'Q10', 'Q11a', 'Q11b',
             'Q12', 'Q13', 'Q14', 'Q15a', 'Q15b', 'Q15c',
             'Q15d', 'Q15e', 'Q16a', 'Q16b', 'Q16c', 'Q16d',
             'Q17', 'Q17a1', 'Q17a2', 'Q17a3', 'Q17a4', 'Q17a5',
             'Q17a6', 'Q17a7', 'Q17a8', 'Q17b', 'Q18', 'Q19',
             'Q20', 'Q21', 'Q22a', 'Q22b', 'Q22c', 'Q22d', 'Q22e',
             'Q22f', 'Q23a', 'Q23b', 'Q23c', 'Q24', 'Q25',
             'Q26a', 'Q26b', 'Q27a', 'Q27b', 'Q27c', 'Q27d',
             'Q27e', 'Q27f', 'Q27g', 'Q27h', 'Q27i', 'Q27j',
             'Q27k', 'Q27l', 'Q27m', 'Yes', 'Q28', 'Q29', 'Q30',
             'Q31', 'yes', 'Q32', 'Q33a', 'Q33b', 'Q34', 'Q35','wt']

LAND23_SCHEMA = ['IDNO','Q1a', 'Q1b', 'Q1c', 'Q1d', 'Q1e', 'Q1f',
             'Q2a', 'Q2b', 'Q2c', 'Q2d', 'Q2e',
             'Q3', 'Q4', 'Q5', 'Q6', 'Q7a', 'Q7b',
             'Q8a', 'Q8b', 'Q9', 'Q10', 'Q11a', 'Q11b',
             'Q12', 'Q13', 'Q14', 'Q15a', 'Q15b', 'Q15c',
             'Q15d', 'Q15e', 'Q16a', 'Q16b', 'Q16c', 'Q16d',
             'Q17', 'Q17a1', 'Q17a2', 'Q17a3', 'Q17a4', 'Q17a5',
             'Q17a6', 'Q17a7', 'Q17a8', 'Q17b', 'Q18', 'Q19',
             'Q20', 'Q21', 'Q22a', 'Q22b', 'Q22c', 'Q22d', 'Q22e',
             'Q22f', 'Q23a', 'Q23b', 'Q23c', 'Q24', 'Q25',
             'Q26a', 'Q26b', 'Q27a', 'Q27b', 'Q27c', 'Q27d',
             'Q27e', 'Q27f', 'Q27g', 'Q27h', 'Q27i', 'Q27j',
             'Q27k', 'Q27l', 'Q27m', 'Q28', 'Q29', 'Q30',
             'Q31', 'Q32', 'Q33', 'yes', 'Q34a', 'Q34b', 'Q35', 'Q36','wt']

WOLF23_SCHEMA = ['IDNO','Q1a', 'Q1b', 'Q1c', 'Q1d', 'Q1e', 'Q1f',
             'Q2a', 'Q2b', 'Q2c', 'Q2d', 'Q2e',
             'Q3', 'Q4', 'Q5', 'Q6', 'Q7a', 'Q7b',
             'Q8a', 'Q8b', 'Q9', 'Q10', 'Q11a', 'Q11b',
             'Q12', 'Q13', 'Q14', 'Q15a', 'Q15b', 'Q15c',
             'Q15d', 'Q15e', 'Q16a', 'Q16b', 'Q16c', 'Q16d',
             'Q17', 'Q17a1', 'Q17a2', 'Q17a3', 'Q17a4', 'Q17a5',
             'Q17a6', 'Q17a7', 'Q17a8', 'Q17b', 'Q18', 'Q19',
             'Q20', 'Q21', 'Q22a', 'Q22b', 'Q22c', 'Q22d', 'Q22e',
             'Q22f', 'Q23a', 'Q23b', 'Q23c', 'Q24', 'Q25',
             'Q26a', 'Q26b', 'Q27a', 'Q27b', 'Q27c', 'Q27d',
             'Q27e', 'Q27f', 'Q27g', 'Q27h', 'Q27i', 'Q27j',
             'Q27k', 'Q27l', 'Q27m', 'yes', 'yes', 'NA', 'Q28', 'Q29',
             'Q30', 'Q31', 'Q32a', 'Q32b', 'Q33', 'Q34','wt']

HOUSE17_SCHEMA = ['IDNO','NA', 'Q12b', 'Q12a', 'NA', 'Q12c', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
              'Q1', 'Na', 'Q2', 'Q3', 'Q4a', 'Q4b', 'NA', 'NA', 'Q5', 'Q6', 'Q7a',
              'Q7b', 'Q8', 'Q9', 'Q10', 'Q11a', 'Q11b', 'Q11c', 'Q11d', 'Q11e',
              'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
              'NA','NA', 'Q18', 'Q19', 'Q16', 'Q17', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'Q20a',
              'Q20b', 'Q20c', 'Q13', 'Q14', 'Q15a', 'Q15b', 'Q26a', 'Q26b', 'Q26c',
              'Q26d', 'Q26e', 'Q26f', 'Q26g', 'Q26h', 'Q26i', 'Q26j', 'Q26k', 'Q26l',
              'Q26m', 'NA', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'NA', 'Q27a', 'Q27b', 'Q28', 'Q29','wt']

DEER17_SCHEMA = ['IDNO','NA', 'Q12b', 'Q12a', 'NA', 'Q12c', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'Q1', 'Na', 'Q2', 'Q3', 'Q4a', 'Q4b', 'NA', 'NA', 'Q5', 'Q6', 'Q7a', 'Q7b',
             'Q8', 'Q9', 'Q10', 'Q11a', 'Q11b', 'Q11c', 'Q11d', 'Q11e', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'Q18', 'Q19', 'Q16', 'Q17', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'Q20a', 'Q20b',
             'Q20c', 'Q13', 'Q14', 'Q15a', 'Q15b', 'Q25a', 'Q25b', 'Q25c', 'Q25d',
             'Q25e', 'Q25f', 'Q25g', 'Q25h', 'Q25i', 'Q25j', 'Q25k', 'Q25l', 'Q25m',
             'NA', 'Q21', 'Q22', 'Q23', 'Q24', 'Yes', 'NA', 'Q26a', 'Q26b', 'Q27', 'Q28','wt']

LAND17_SCHEMA = ['IDNO','NA', 'Q12b', 'Q12a', 'NA', 'Q12c', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'Q1', 'Na', 'Q2', 'Q3', 'Q4a', 'Q4b', 'NA', 'NA', 'Q5', 'Q6', 'Q7a', 'Q7b',
             'Q8', 'Q9', 'Q10', 'Q11a', 'Q11b', 'Q11c', 'Q11d', 'Q11e', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'Q18', 'Q19', 'Q16', 'Q17', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'Q20a', 'Q20b',
             'Q20c', 'Q13', 'Q14', 'Q15a', 'Q15b', 'Q26a', 'Q26b', 'Q26c', 'Q26d',
             'Q26e', 'Q26f', 'Q26g', 'Q26h', 'Q26i', 'Q26j', 'Q26k', 'Q26l', 'Q26m',
             'NA', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'NA', 'Q27a', 'Q27b', 'Q28', 'Q29','wt']

WOLF17_SCHEMA = ['IDNO','NA', 'Q12b', 'Q12a', 'NA', 'Q12c', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'Q1', 'Na', 'Q2', 'Q3', 'Q4a', 'Q4b', 'NA', 'NA', 'Q5', 'Q6', 'Q7a', 'Q7b',
             'Q8', 'Q9', 'Q10', 'Q11a', 'Q11b', 'Q11c', 'Q11d', 'Q11e', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'Q18', 'Q19', 'Q16', 'Q17', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'Q20a', 'Q20b',
             'Q20c', 'Q13', 'Q14', 'Q15a', 'Q15b', 'Q25a', 'Q25b', 'Q25c', 'Q25d',
             'Q25e', 'Q25f', 'Q25g', 'Q25h', 'Q25i', 'Q25j', 'Q25k', 'Q25l', 'Q25m',
             'Yes', 'Yes', 'Q21', 'Q22', 'Q23', 'Q24', 'NA', 'Q26a', 'Q26b', 'Q27', 'Q28','wt']

HOUSE12_SCHEMA = ['IDNO','NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
              'q1', 'Na', 'q5', 'q7b', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
              'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
              'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'q14',
              'q13', 'q3', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'q2a', 'q2b', 'q2c',
              'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
              'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'q9', 'q10', 'NA', 'NA',  'q15','NA', 'q17a', 'q17b',
              'q18', 'q19','wt']

DEER12_SCHEMA = ['IDNO','NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'q1', 'Na', 'q5', 'q7b', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'q14',
             'q13', 'q3', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'q2a', 'q2b', 'q2c',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA','NA', 'q9', 'q10', 'NA','NA','yes', 'NA', 'q16a', 'q16b',
             'q17', 'q18','wt']

LAND12_SCHEMA = ['IDNO','NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'q1', 'Na', 'q5', 'q7b', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'q14', 'q13', 'q3',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'q2a', 'q2b', 'q2c', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA','NA','q9', 'q10', 'NA', 'NA', 'q15', 'NA', 'q17a', 'q17b', 'q18', 'q19','wt']

WOLF12_SCHEMA = ['IDNO','NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'q1', 'Na', 'q5', 'q7b', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA','NA', 'q14', 'q13', 'q3',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'q2a', 'q2b', 'q2c', 'NA', 'NA',
             'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA',
             'NA', 'NA', 'NA','NA', 'yes', 'q10', 'NA', 'NA', 'q15', 'NA', 'q17a', 'q17b', 'q18', 'q19','wt']


In [32]:
SCHEMAS = {
"2023_HOUSE": HOUSE23_SCHEMA,
"2023_DEER":  DEER23_SCHEMA,
"2023_LAND":  LAND23_SCHEMA,
"2023_WOLF":  WOLF23_SCHEMA,

"2017_HOUSE": HOUSE17_SCHEMA,
"2017_DEER":  DEER17_SCHEMA,
"2017_LAND":  LAND17_SCHEMA,
"2017_WOLF":  WOLF17_SCHEMA,

"2012_HOUSE": HOUSE12_SCHEMA,
"2012_DEER":  DEER12_SCHEMA,
"2012_LAND":  LAND12_SCHEMA,
"2012_WOLF":  WOLF12_SCHEMA,
}

In [33]:
def _case_insensitive_get(df: pd.DataFrame, name: str):
    cols_lower = {c.lower(): c for c in df.columns}
    return df[cols_lower[name.lower()]] if name.lower() in cols_lower else None

def to_house23_by_schema(df: pd.DataFrame,
                         schema_list: list[str],
                         house23_cols: list[str],
                         group: str,
                         year: int) -> pd.DataFrame:
    assert len(schema_list) == len(house23_cols), "Schema length must match House23"
    src = df.copy()
    src.columns = [str(c).strip() for c in src.columns]

    out = pd.DataFrame(index=src.index)
    missing_sources = []

    for tgt_col, token in zip(house23_cols, schema_list):
        t = (token or "").strip()
        if t == "" or t.upper() == "NA":
            out[tgt_col] = np.nan
        elif t.lower() == "yes":
            out[tgt_col] = 2
        else:
            col = _case_insensitive_get(src, t)
            if col is None:
                out[tgt_col] = np.nan
                missing_sources.append(t)
            else:
                out[tgt_col] = col

    out["group"] = group
    out["year"] = year

    # Helpful warning for you while building schemas:
    if missing_sources:
        print(f"[WARN] Missing source columns for {year}_{group}: {sorted(set(missing_sources))}")

    return out

def infer_year_group(filename: str):
    n = filename.lower()
    # Map 2016 files to 2017 wave (as in your R)
    year = 2012 if "2012" in n else 2017 if ("2016" in n or "2017" in n) else 2023
    if "household" in n or "households" in n or "grey" in n:
        group = "HOUSE"
    elif "deer elk" in n or "deer" in n or "green" in n:
        group = "DEER"
    elif "landowner" in n or "yellow" in n:
        group = "LAND"
    else:
        group = "WOLF"
    return year, group

In [34]:
FOLDER = r"D:\MontanaBackup\WolfData\WolfDatasets"

file_names = [
    "Final Respondents Deer Elk License Holders Wolf Survey 2016.csv",
    "Final Respondents Private Landowner Wolf Survey 2016.csv",
    "Final Respondents Wolf License Holders Wolf Survey 2016.csv",
    "Full Household Results 2017.csv",
    "Households 2012 Wolf Hunt Survey Respondents.csv",
    "FINAL Quantitative Respondent Data for Wolf Deer Elk Hunter Survey 2023.csv",
    "FINAL Quantitative Respondent Data for Wolf Hunter Wolf Survey 2023.csv",
    "FINAL Quantitative Respondent Data for Wolf Private Landowner Survey 2023.csv",
    "FINAL Quantitative Respondent Data for the Wolf Household Survey 2023.csv",
    "Resident Deer Elk License Holders 2012 Wolf Hunt Survey Results.csv",
    "Resident Private Landowners 2012 Wolf Hunt Survey Results.csv",
    "Resident Wolf License Holders 2012 Wolf Hunt Survey Results.csv",
]

paths = [os.path.join(FOLDER, f) for f in file_names]

In [35]:
frames = []

for p in paths:
    raw = pd.read_csv(p)
    year, group = infer_year_group(os.path.basename(p))
    key = f"{year}_{group}"

    if key not in SCHEMAS:
        print(f"[SKIP] No schema registered for {key} (file: {os.path.basename(p)})")
        continue

    schema = SCHEMAS[key]
    df_h23 = to_house23_by_schema(
        df=raw,
        schema_list=schema,
        house23_cols=HOUSE23,
        group=group,
        year=year,
    )
    frames.append(df_h23)

MasterData = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame(columns=HOUSE23 + ["group","year"])
print(MasterData.shape)
MasterData.head()

[WARN] Missing source columns for 2023_DEER: ['wt']
[WARN] Missing source columns for 2023_WOLF: ['wt']
[WARN] Missing source columns for 2023_LAND: ['wt']
[WARN] Missing source columns for 2023_HOUSE: ['wt']
(7607, 90)


,IDNO,Q1a,Q1b,Q1c,Q1d,Q1e,Q1f,Q2a,Q2b,Q2c,Q2d,Q2e,Q3,Q4,Q5,Q6,Q7a,Q7b,Q8a,Q8b,Q9,Q10,Q11a,Q11b,Q12,Q13,Q14,Q15a,Q15b,Q15c,Q15d,Q15e,Q16a,Q16b,Q16c,Q16d,Q17,Q17a1,Q17a2,Q17a3,Q17a4,Q17a5,Q17a6,Q17a7,Q17a8,Q17b,Q18,Q19,Q20,Q21,Q22a,Q22b,Q22c,Q22d,Q22e,Q22f,Q23a,Q23b,Q23c,Q24,Q25,Q26a,Q26b,Q27a,Q27b,Q27c,Q27d,Q27e,Q27f,Q27g,Q27h,Q27i,Q27j,Q27k,Q27l,Q27m,Q28,Q29,Q30,Q31,Q32,Q33,Q34,Q35a,Q35b,Q36,Q37,wt,group,year
0,1.0,NaN,3.0,3.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,5.0,1.0,NaN,NaN,NaN,5.0,5.0,1.0,NaN,3.0,3.0,3.0,2.0,2.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,5.0,5.0,1.0,NaN,1.0,3.0,5.0,1.0,3.0,1.0,5.0,1.0,1.0,1.0,7.0,4.0,7.0,NaN,1.0,1.0,1.0,1.0,2.0,NaN,1.0,NaN,55.0,2.0,1.0,DEER,2017
1,2.0,NaN,3.0,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,1.0,NaN,NaN,NaN,3.0,5.0,1.0,NaN,3.0,3.0,1.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,5.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0,5.0,2.0,1.0,7.0,1.0,2.0,1.0,7.0,2.0,4.0,7.0,1.0,2.0,7.0,3.0,7.0,NaN,1.0,5.0,1.0,1.0,2.0,NaN,1.0,NaN,70.0,2.0,1.0,DEER,2017
2,4.0,NaN,2.0,5.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.0,4.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,3.0,5.0,3.0,3.0,3.0,5.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1.0,5.0,2.0,1.0,NaN,4.0,4.0,6.0,1.0,6.0,1.0,4.0,1.0,7.0,4.0,1.0,4.0,7.0,NaN,1.0,1.0,1.0,1.0,2.0,NaN,1.0,NaN,39.0,2.0,1.0,DEER,2017
3,5.0,NaN,1.0,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,1.0,4.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,63.0,1.0,1.0,DEER,2017
4,6.0,NaN,1.0,2.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.0,4.0,1.0,NaN,NaN,NaN,4.0,3.0,1.0,NaN,3.0,3.0,3.0,5.0,3.0,5.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,4.0,5.0,1.0,NaN,3.0,2.0,2.0,2.0,5.0,5.0,2.0,4.0,1.0,2.0,7.0,2.0,7.0,NaN,1.0,5.0,1.0,1.0,2.0,NaN,1.0,NaN,31.0,2.0,1.0,DEER,2017


In [36]:
out_path = os.path.join(FOLDER, "MasterDataGenerate_python.csv")
MasterData.to_csv(out_path, index=False)
out_path

'D:\\MontanaBackup\\WolfData\\WolfDatasets\\MasterDataGenerate_python.csv'